In [1]:
# --------------------------
# 1. Data Preprocessing (Corrected)
# --------------------------
# Split data first
train_size = int(len(data) * 0.8)
train_raw, test_raw = data.iloc[:train_size], data.iloc[train_size:]

# Scale using only training data
scaler = MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train_raw), 
                           columns=data.columns, 
                           index=train_raw.index)
test_scaled = pd.DataFrame(scaler.transform(test_raw), 
                          columns=data.columns, 
                          index=test_raw.index)

# Ensure no NaN values
train = train_scaled.dropna()
test = test_scaled.dropna()

# --------------------------
# 2. ARIMA Model (Remains Same)
# --------------------------
arima_model = ARIMA(train["Price"], order=(5,1,0)).fit()
arima_pred = arima_model.forecast(steps=len(test))

# --------------------------
# 3. GARCH Model (Volatility Prediction - Not Evaluated)
# --------------------------
# This section remains but isn't used in price evaluation
scaled_price = train["Price"] * 10
garch_model = arch_model(scaled_price, vol='Garch', p=1, q=1, rescale=False).fit()

# --------------------------
# 4. VAR Model (Correct Column Selection)
# --------------------------
var_model = VAR(train)
var_fitted = var_model.fit(5)
var_pred = var_fitted.forecast(train.values[-5:], steps=len(test))

# --------------------------
# 5. Markov-Switching Model (Unchanged)
# --------------------------
markov_model = MarkovRegression(train["Price"], k_regimes=2, trend='c', switching_variance=True).fit()

# --------------------------
# 6. LSTM Model (Unchanged)
# --------------------------
# ... [LSTM implementation remains the same] ...

# --------------------------
# 7. Model Evaluation (Corrected)
# --------------------------
def evaluate_model(actual, predicted, model_name):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    print(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(test["Price"], arima_pred, "ARIMA")
evaluate_model(test["Price"], var_pred[:,0], "VAR")  # Ensure first column is Price
evaluate_model(test["Price"].values[1:], lstm_pred.flatten(), "LSTM")

# --------------------------
# 8. Visualization (Adjusted)
# --------------------------
plt.figure(figsize=(12,6))
plt.plot(test.index, test["Price"], label="Actual Price", color='black')
plt.plot(test.index, arima_pred, label="ARIMA", linestyle='dashed')
plt.plot(test.index, var_pred[:,0], label="VAR", linestyle='dashed')
plt.plot(test.index[1:], lstm_pred, label="LSTM", linestyle='dashed')
plt.legend()
plt.title("Model Predictions vs Actual Price")
plt.show()

NameError: name 'data' is not defined